# Entrenamiento de modelos YOLO

Entrenamiento de las versiones n, s y m de YOLOv8 para deteccion de lesiones en imagenes del dataset ISIC 2019.

In [1]:
from ultralytics import YOLO
import pandas as pd
import matplotlib.pyplot as plt
from pathlib import Path
import yaml
import shutil
from PIL import Image

## Preparacion del entorno

Importacion de librerias necesarias para el entrenamiento y visualizacion.

In [2]:
def preparar_estructura_yolo():
    base_dir = Path('yolo_dataset')
    
    for split in ['train', 'val']:
        (base_dir / split / 'images').mkdir(parents=True, exist_ok=True)
        (base_dir / split / 'labels').mkdir(parents=True, exist_ok=True)
    
    return base_dir

## Estructura de datos YOLO

Creacion de la estructura de carpetas requerida por YOLO para entrenamiento y validacion.

In [3]:
def crear_anotaciones_yolo(csv_path, output_dir, split_name):
    df = pd.read_csv(csv_path)
    img_dir = Path('conjunto_de_datos/ISIC_2019_Training_Input')
    
    for _, row in df.iterrows():
        img_name = row['image'] + '.jpg'
        img_path = img_dir / img_name
        
        if img_path.exists():
            dest_img = output_dir / split_name / 'images' / img_name
            shutil.copy(img_path, dest_img)
            
            img = Image.open(img_path)
            w, h = img.size
            
            x_center = 0.5
            y_center = 0.5
            width = 0.8
            height = 0.8
            
            label_path = output_dir / split_name / 'labels' / (row['image'] + '.txt')
            with open(label_path, 'w') as f:
                f.write(f'0 {x_center} {y_center} {width} {height}\n')

## Generacion de dataset

Copia de imagenes y generacion de anotaciones en formato YOLO a partir de los subconjuntos CSV.

In [4]:
base_dir = preparar_estructura_yolo()

crear_anotaciones_yolo('conjunto_de_datos/train1_subset.csv', base_dir, 'train')
crear_anotaciones_yolo('conjunto_de_datos/test1_subset.csv', base_dir, 'val')

data_yaml = {
    'path': str(base_dir.absolute()),
    'train': 'train/images',
    'val': 'val/images',
    'nc': 1,
    'names': ['lesion']
}

yaml_path = base_dir / 'data.yaml'
with open(yaml_path, 'w') as f:
    yaml.dump(data_yaml, f)

## Configuracion de modelos

Definicion de los tres modelos YOLO a entrenar: nano, small y medium.

In [5]:
modelos = ['yolov8n', 'yolov8s', 'yolov8m']
resultados = {}

## Entrenamiento

Ejecucion del entrenamiento para cada modelo con parametros optimizados para CPU: 15 epocas, imagenes de 320px, batch size 4 y un worker.

In [6]:
for modelo_nombre in modelos:
    modelo = YOLO(f'{modelo_nombre}.pt')
    
    resultados_train = modelo.train(
        data=str(yaml_path),
        epochs=15,
        imgsz=320,
        batch=4,
        name=modelo_nombre,
        patience=5,
        save=True,
        device='cpu',
        workers=1
    )
    
    resultados[modelo_nombre] = resultados_train

New https://pypi.org/project/ultralytics/8.3.236 available 😃 Update with 'pip install -U ultralytics'
Ultralytics 8.3.235 🚀 Python-3.12.1 torch-2.9.1+cu128 CPU (Intel Xeon Platinum 8370C CPU @ 2.80GHz)
engine/trainer: agnostic_nms=False, amp=True, augment=False, auto_augment=randaugment, batch=4, bgr=0.0, box=7.5, cache=False, cfg=None, classes=None, close_mosaic=10, cls=0.5, compile=False, conf=None, copy_paste=0.0, copy_paste_mode=flip, cos_lr=False, cutmix=0.0, data=yolo_dataset/data.yaml, degrees=0.0, deterministic=True, device=cpu, dfl=1.5, dnn=False, dropout=0.0, dynamic=False, embed=None, epochs=15, erasing=0.4, exist_ok=False, fliplr=0.5, flipud=0.0, format=torchscript, fraction=1.0, freeze=None, half=False, hsv_h=0.015, hsv_s=0.7, hsv_v=0.4, imgsz=320, int8=False, iou=0.7, keras=False, kobj=1.0, line_width=None, lr0=0.01, lrf=0.01, mask_ratio=4, max_det=300, mixup=0.0, mode=train, model=yolov8n.pt, momentum=0.937, mosaic=1.0, multi_scale=False, name=yolov8n4, nbs=64, nms=False

## Visualizacion de metricas

Funcion para generar graficas de las metricas de entrenamiento: loss, precision, recall y mAP. Estas graficas permiten identificar sobreajuste cuando las curvas de train y validacion divergen significativamente.

In [7]:
def graficar_metricas(modelo_nombre):
    results_path = Path(f'runs/detect/{modelo_nombre}/results.csv')
    
    if not results_path.exists():
        return
    
    df = pd.read_csv(results_path)
    df.columns = df.columns.str.strip()
    
    fig, axes = plt.subplots(2, 2, figsize=(15, 10))
    fig.suptitle(f'Metricas de entrenamiento - {modelo_nombre}', fontsize=16)
    
    axes[0, 0].plot(df['epoch'], df['train/box_loss'], label='Train Box Loss')
    axes[0, 0].plot(df['epoch'], df['val/box_loss'], label='Val Box Loss')
    axes[0, 0].set_xlabel('Epoch')
    axes[0, 0].set_ylabel('Loss')
    axes[0, 0].set_title('Box Loss')
    axes[0, 0].legend()
    axes[0, 0].grid(True)
    
    axes[0, 1].plot(df['epoch'], df['train/cls_loss'], label='Train Cls Loss')
    axes[0, 1].plot(df['epoch'], df['val/cls_loss'], label='Val Cls Loss')
    axes[0, 1].set_xlabel('Epoch')
    axes[0, 1].set_ylabel('Loss')
    axes[0, 1].set_title('Classification Loss')
    axes[0, 1].legend()
    axes[0, 1].grid(True)
    
    axes[1, 0].plot(df['epoch'], df['metrics/precision(B)'], label='Precision')
    axes[1, 0].plot(df['epoch'], df['metrics/recall(B)'], label='Recall')
    axes[1, 0].set_xlabel('Epoch')
    axes[1, 0].set_ylabel('Score')
    axes[1, 0].set_title('Precision y Recall')
    axes[1, 0].legend()
    axes[1, 0].grid(True)
    
    axes[1, 1].plot(df['epoch'], df['metrics/mAP50(B)'], label='mAP50')
    axes[1, 1].plot(df['epoch'], df['metrics/mAP50-95(B)'], label='mAP50-95')
    axes[1, 1].set_xlabel('Epoch')
    axes[1, 1].set_ylabel('mAP')
    axes[1, 1].set_title('Mean Average Precision')
    axes[1, 1].legend()
    axes[1, 1].grid(True)
    
    plt.tight_layout()
    plt.savefig(f'metricas_{modelo_nombre}.png', dpi=300, bbox_inches='tight')
    plt.show()

Generacion de graficas individuales para cada modelo entrenado.

In [8]:
for modelo_nombre in modelos:
    graficar_metricas(modelo_nombre)

<Figure size 1500x1000 with 4 Axes>

<Figure size 1500x1000 with 4 Axes>

## Comparacion entre modelos

Graficas comparativas que muestran el rendimiento relativo de los tres modelos en terminos de loss y mAP.

In [9]:
def comparar_modelos():
    fig, axes = plt.subplots(1, 2, figsize=(15, 5))
    fig.suptitle('Comparacion entre modelos YOLO', fontsize=16)
    
    for modelo_nombre in modelos:
        results_path = Path(f'runs/detect/{modelo_nombre}/results.csv')
        
        if results_path.exists():
            df = pd.read_csv(results_path)
            df.columns = df.columns.str.strip()
            
            axes[0].plot(df['epoch'], df['val/box_loss'], label=modelo_nombre)
            axes[1].plot(df['epoch'], df['metrics/mAP50(B)'], label=modelo_nombre)
    
    axes[0].set_xlabel('Epoch')
    axes[0].set_ylabel('Validation Box Loss')
    axes[0].set_title('Loss de validacion')
    axes[0].legend()
    axes[0].grid(True)
    
    axes[1].set_xlabel('Epoch')
    axes[1].set_ylabel('mAP50')
    axes[1].set_title('Mean Average Precision')
    axes[1].legend()
    axes[1].grid(True)
    
    plt.tight_layout()
    plt.savefig('comparacion_modelos.png', dpi=300, bbox_inches='tight')
    plt.show()

comparar_modelos()

<Figure size 1500x500 with 2 Axes>

## Guardado de modelos

Copia de los mejores pesos de cada modelo entrenado a una carpeta centralizada.

In [10]:
Path('modelos_entrenados').mkdir(exist_ok=True)

for modelo_nombre in modelos:
    weight_path = Path(f'runs/detect/{modelo_nombre}/weights/best.pt')
    if weight_path.exists():
        dest = Path('modelos_entrenados') / f'{modelo_nombre}_best.pt'
        shutil.copy(weight_path, dest)